# Explanation of pairs code indexing 

A good way to explain pairs is to consider that it is a special case of combinations. In combinations, we iterate over the entire length of the array for both the `left` and `right` arrays. So, if we consider the iteration indices `i` and `j` as the two dimensions of a matrix, then we are iterating over all elements of the matrix $C = \{c_{ij}\} \quad \forall \: i,j \in [0,len]$. So `left[i*len+j] = i` and `right[i*len+j] = j`.

### Pairs as upper triangular version of combinations

In pairs, we change that. We remove redundant pairs of values. This amounts to considering the upper triangular matrix $C_{u}$ of $C$. If we now manage to iterate `i` and `j` following the upper triangular construct, we will form the pairs index. This exactly what the sequential algorithm for pairs is doing:

```python
for i in range(base_len-1):
    pairs_i = pairs_lengths[i]
    for j in range(start[i], stop[i]):
        for k in range(j+1, stop[i]):
            left[pairs_i] = j
            right[pairs_i] = k
            pairs_i +=1
```
Note that how k is iterating from `j+1` to `stop`, not from `start` to `stop`. This affirms the notion of upper triangular matrix.

But in CUDA, we cannot just iterate over so easily. We need a procedure to allow us to generate the indices parallely. For simplicity, consider the single event case in **test_pairs_indices** notebook. Let `j` denote the rows, and `k` denote the columns of the matrix $C$. We want to eliminate the lower half of the matrix. 

For this part, let me first write out the index of left and right arrays: 

```cpp
len*(len+1)/2-(j*(j+1)/2)-k-1
```

Note that the pairs arrays `left` and `right` will have $ \frac{len*(len+1)}{2}$ elements,  if we are starting from 0. This explains the first element in index above.

### Lower triangular matrix indices 

We shall now relate the particle index with it's row index j and column index k in lower triangular matrix. Well, that's easy. The particle index at row j and column k if given by $\frac{j*(j+1)}{2} + k +1$. It can also be derived as follows:

For any row j>1, we have already counted $ 1+2+3+ \ldots + j = \frac{j*(j+1)}{2}$ particles ( lower triangular matrix elements). So the element at row `j` starts at index $\frac{j*(j+1)}{2}+1$. So, an element at column k of that row will be at index $\frac{j*(j+1)}{2}+1+k$.

### Generation of upper ttraingular matrix indices

However, our aim is to index upper traingular matrix way. To do that, we now proceed bottom up from last element in the upper-triangular matrix. The last element will have index $ \frac{len*(len+1)}{2}-1$. The element previous to that will have $ \frac{len*(len+1)}{2}-2$ and so on. So an element at row `k` and column `j` ( note the inversion of j and k) will have index $ \frac{len*(len+1)}{2} -(\frac{j*(j+1)}{2}+1+k) $. Note the second term is the particle index in lower triangular matrix. This is what leads us to index as given above.

### Values of arrays `left` and `right`
Now as for the values of left and right arrays, we note that we would have normally assigned j and to them. However, as we are proceeding bottom up, so, we instead assign them the values len-j-1 and len-k-1,if you want right to start from 0 and not 1.

Thus, combining all above

```cpp
left[len*(len+1)/2-(j*(j+1)/2)-k-1] = len-1-j;
right[len*(len+1)/2-(j*(j+1)/2)-k-1] = len-k;
```

The full code is:
```cpp
int i = threadIdx.x + blockDim.x*blockIdx.x;
int j = threadIdx.y + blockDim.y*blockIdx.y;
int k =  blockIdx.z*blockDim.z+threadIdx.z;
if ( i<length[0] && j< lengths[i] && k<=j) 
{
     left[pairs_lengths[i]+lengths[i]*(lengths[i]+1)/2-(j*(j+1)/2)-k-1] = lengths[i]-1-j+start[i];
     right[pairs_lengths[i]+lengths[i]*(lengths[i]+1)/2 - (j*(j+1)/2) -k -1] = lengths[i]-k-1 + start[i];
    }
```

The `pairs_lengths[i]` and `start[i]` just add offsets to the arrays for different events `i`.

